In [4]:
# k = matriz de rigidez local
# a = matriz de compatibilidad geometrica

# K = matriz de rigidez global
# a^t*k*a = K

# F = vector de fuerzas

# Epsilon = [teta_a teta_b delta]
# sigma_p = [Ma Mb F]

# T = matriz de traslacion
# Kt = matriz de rigidez total

# sumatoria de (T^t*K*T)_i = Kt

# Vector de deformaciones y giros = r
# Kt*F = r

# a*r = E
# k*E = sigma

1. Definir: $k_i$, $S$, $a_i$, $T_i$
    - $k_i$: rigidez local de cada barra.
    - $S$: vector de solicitaciones en los nodos.
    - $a_i$: matriz de compatibilidad geometrica de cada barra.
    - $T_i$: matriz de traslacion de cada barra.


2. Rigidez global: $$ a_i^t k_i a_i = K_i $$
- Rigidez total (del sistema): $$ \sum T_i^t K_i T_i = K_t $$
- Desplazamientos nodales: $$ K_t^{-1} \vec S = \vec r $$
- Deformaciones: $$ a_i \vec r_i = \vec \epsilon_i $$
- Esfuerzos: $$ k_i \vec \epsilon_i = \vec \sigma_i $$

In [5]:
#input
i = 90 # grados
b = 30 # [cm]
h = 40 # [cm]
L = 300 # [cm]
ϰ = 0 # kappa
resistencia = 19.5 # [MPa]
𝜈 = 0.25; # modulo de poisson

# printable info
A = b*h # [cm²]
E = 15100*√(resistencia*1000)/100 # [kg/mm²]
I = b*h^3/12 # [cm⁴]

# _internals_
G = E/(2(1+𝜈)) # modulo de cizalle
β = 6*E*I*ϰ/(G*A*L^2);

𝛾 = i*π/180 # [rad]
E = E/0.01 # [kg/cm²]
s = round(sin(𝛾), digits=4)
c = round(cos(𝛾), digits=4)

#= ANALISIS Matricial =#
k11 = 2E*I*(2 + β)/(L*(1 + 2β))
k12 = 2E*I*(1 - β)/(L*(1 + 2β))
k33 = A*E/L

k1 = [
    k11 k12 0;
    k12 k11 0;
    0   0   k33;
] # rigidez local

a1 = [
    -s/L c/L 1  s/L -c/L 0;
    -s/L c/L 0  s/L -c/L 1;
     c   s   0 -c   -s   0;
] # compatibilidad geometrica

K1 = a1'k1*a1;

In [6]:
#input
i = 0 # grados
b = 30 # [cm]
h = 40 # [cm]
L = 200 # [cm]
ϰ = 0 # kappa
resistencia = 19.5 # [MPa]
𝜈 = 0.25; # modulo de poisson

# printable info
A = b*h # [cm²]
E = 15100*√(resistencia*1000)/100 # [kg/mm²]
I = b*h^3/12 # [cm⁴]

# _internals_
G = E/(2(1+𝜈)) # modulo de cizalle
β = 6*E*I*ϰ/(G*A*L^2);

𝛾 = i*π/180 # [rad]
E = E/0.01 # [kg/cm²]
s = round(sin(𝛾), digits=4)
c = round(cos(𝛾), digits=4)

#= ANALISIS Matricial =#
k11 = 2E*I*(2 + β)/(L*(1 + 2β))
k12 = 2E*I*(1 - β)/(L*(1 + 2β))
k33 = A*E/L

k3 = [
    k11 k12 0;
    k12 k11 0;
    0   0   k33;
] # rigidez local

a3 = [
    -s/L c/L 1  s/L -c/L 0;
    -s/L c/L 0  s/L -c/L 1;
     c   s   0 -c   -s   0;
] # compatibilidad geometrica

K3 = a3'k3*a3;

In [7]:
F = [10000; -10000; 0; 10000; -10000; 0;];

T1 =[
    0 0 0 0 0 0;
    0 0 0 0 0 0;
    0 0 0 0 0 0;
    1 0 0 0 0 0;
    0 1 0 0 0 0;
    0 0 1 0 0 0;
]

T2 =[
    0 0 0 0 0 0;
    0 0 0 0 0 0;
    0 0 0 0 0 0;
    0 0 0 1 0 0;
    0 0 0 0 1 0;
    0 0 0 0 0 1;
]

T3 =[
    1 0 0 0 0 0;
    0 1 0 0 0 0;
    0 0 1 0 0 0;
    0 0 0 1 0 0;
    0 0 0 0 1 0;
    0 0 0 0 0 1;
]

Kt = T1'*K1*T1 + T2'*K1*T2 + T3'*K3*T3
r = Kt^-1*F

6-element Array{Float64,1}:
  0.0888336817855281
  0.00039598792540071747
 -0.00014761680059316245
  0.08883368178552811
 -0.00276722923016705
 -0.0001476168005931625

In [8]:
r1 = [0; 0; 0; r[1:3]]
r2 = [0; 0; 0; r[4:6]]
r3 = r

ϵ1 = a1*r1
ϵ2 = a1*r2
ϵ3 = a3*r3

σ1 = k1*ϵ1
σ2 = k1*ϵ2
σ3 = k3*ϵ3

σ = [
    round(σ1[1]/100) round(σ2[1]/100) round(σ3[1]/100);
    round(σ1[2]/100) round(σ2[2]/100) round(σ3[2]/100);
    round(σ1[3]) round(σ2[3]) round(σ3[3]);
]

V = [(σ[1,1] + σ[2,1])/3 (σ[1,2] + σ[2,2])/3 (σ[1,3] + σ[2,3])/2]

println("Ma: ", σ[1,:])
println("Mb: ", σ[2,:])
println("F:  ", σ[3,:])
println("V:  ", V)

Ma: [16660.0, 16660.0, -13340.0]
Mb: [13340.0, 13340.0, -13340.0]
F:  [-3340.0, 23340.0, -0.0]
V:  [10000.0 10000.0 -13340.0]
